In [ ]:
#export
from local.imports import *
from local.test import *
from local.core import *
from local.data.transform import *
from local.data.core import *
from local.data.source import *
from local.data.external import *
from local.data.pipeline import *
from local.text.core import *
from local.notebook.showdoc import show_doc

In [ ]:
#default_exp text.data
#default_cls_lvl 3

# Text data

> Functions and transforms to help gather text data in a `DataSource

## Numericalizing

In [ ]:
def make_vocab(count, min_freq=3, max_vocab=60000):
    "Create a vocab of `max_vocab` size from `Counter` `count` with items present more than `min_freq`"
    vocab = [o for o,c in count.most_common(max_vocab) if c >= min_freq]
    for o in reversed(defaults.text_spec_tok): #Make sure all special tokens are in the vocab
        if o in vocab: vocab.remove(o)
        vocab.insert(0, o)
    vocab = vocab[:max_vocab]
    if len(vocab) < max_vocab and len(vocab)%8 != 0: 
        #Make sure vocab size is a multiple of 8 for fast mixed precision training
        vocab += ['xxfake' for _ in range(0, 8-len(vocab)%8)]
    return vocab

In [ ]:
count = Counter(['a', 'a', 'a', 'a', 'b', 'b', 'c', 'c', 'd'])
test_eq(set(make_vocab(count)), set(defaults.text_spec_tok + 'a xxfake'.split()))
test_eq(len(make_vocab(count))%8, 0)
test_eq(set(make_vocab(count, min_freq=1)), set(defaults.text_spec_tok + 'a b c d xxfake'.split()))
test_eq(set(make_vocab(count,max_vocab=12, min_freq=1)), set(defaults.text_spec_tok + 'a b c'.split()))

In [ ]:
# export
class Numericalize(ItemTransform):
    "Reversible transform of tokenized texts to numericalized ids"
    def __init__(self, vocab=None, min_freq=3, max_vocab=60000, sep=None):
        self.sep = sep or defaults.text_token_sep
        self.vocab,self.min_freq,self.max_vocab = vocab,min_freq,max_vocab
        self.o2i = None if vocab is None else defaultdict(int, {v:k for k,v in enumerate(vocab)})
    
    def setup(self, dsrc):
        if dsrc is None: return
        if self.vocab is None:
            dsrc = getattr(dsrc,'train',dsrc)
            count = Counter(p for o in dsrc for p in o.split(self.sep))
            self.vocab = make_vocab(count, min_freq=self.min_freq, max_vocab=self.max_vocab)
            self.o2i = defaultdict(int, {v:k for k,v in enumerate(self.vocab) if v != 'xxfake'})

    def encodes(self, o):      return [self.o2i[o_] for o_ in o.split(self.sep)]
    def decodes(self, o)->Str: return self.sep.join([self.vocab[o_] for o_ in o if self.vocab[o_] != PAD])

In [ ]:
num = Numericalize(min_freq=1, sep=' ')
num.setup(L('This is an example of text', 'this is another text'))
test_eq(set(num.vocab), set(defaults.text_spec_tok + 'This is an example of text this another xxfake'.split()))
test_eq(len(num.vocab)%8, 0)
start = 'This is an example of text'
t = num(start)
test_eq(t, [11, 9, 12, 13, 14, 10])
test_eq(num.decode(t), start)

In [ ]:
num = Numericalize(min_freq=2, sep=' ')
num.setup(L('This is an example of text', 'this is another text'))
test_eq(set(num.vocab), set(defaults.text_spec_tok + 'is text xxfake'.split()))
test_eq(len(num.vocab)%8, 0)
t = num(start)
test_eq(t, [0, 9, 0, 0, 0, 10])
test_eq(num.decode(t), f'{UNK} is {UNK} {UNK} {UNK} text')

## LM_Dataset -

In [ ]:
#export
class LM_Dataset(GetAttr):
    _xtra = ['show', 'decode', 'show_at', 'decode_at', 'decode_batch']
    def __init__(self, ds, lens=None, bs=64, seq_len=70, shuffle=False, cache=2):
        self.default = self.ds = ReindexCollection(ds, cache=cache)
        self.bs,self.seq_len,self.shuffle = bs,seq_len,shuffle
        if lens is None: lens = [len(o[0]) for o in ds]
        self.lens = ReindexCollection(lens, idxs=self.ds.idxs)
        # The "-1" is to allow for final label
        self.n = round_multiple(sum(lens)-1, bs*seq_len, round_down=True)
        self.cs = self.n//bs   # chunk size
        self.reset()
        
    def reset(self):
        if self.shuffle: self.ds.shuffle()
        self.cum_lens = np.cumsum(self.lens)
        
    def get_loc(self, sl, it, b, xy):
        tokidx = sl*self.seq_len + b*self.cs + it + xy
        docidx = np.searchsorted(self.cum_lens, tokidx+1)
        return self.ds[docidx][0][tokidx-self.cum_lens[docidx]]

    def __len__(self): return self.n//(self.bs*self.seq_len)
    def __getitem__(self, sl):
        if sl>=len(self): raise IndexError
        return [[[self.get_loc(sl, it, b, xy) for it in range(self.seq_len)]
                for b in range(self.bs)] for xy in range(2)]

In [ ]:
bs,sl = 4,3
txts = [[o] for o in 'abcde fghijk lmnopqrs tu vwx y'.split()]

In [ ]:
t = LM_Dataset(txts, bs=bs, seq_len=sl)
test_eq(list(t),
        [[[['a','b','c'], ['g','h','i'], ['m','n','o'], ['s', 't', 'u']],
          [['b','c','d'], ['h','i','j'], ['n','o','p'], ['t','u','v']]],
         [[['d','e','f'], ['j','k','l'], ['p','q','r'], ['v','w','x']],
          [['e','f','g'], ['k','l','m'], ['q','r','s'], ['w','x','y']]]])

In [ ]:
t = LM_Dataset(txts[:-1], bs=bs, seq_len=sl)
test_eq(list(t),
        [[[['a','b','c'], ['d','e','f'], ['g','h','i'], ['j','k','l']],
          [['b','c','d'], ['e','f','g'], ['h','i','j'], ['k','l','m']]]])

## Integration example

In [ ]:
#export
from IPython.display import display, HTML

class TensorText(TensorBase):
    def get_ctxs(self, max_samples=10, **kwargs):
        n_samples = min(self.shape[0], max_samples)
        df = pd.DataFrame({'index': range(n_samples)})
        return [df.iloc[i] for i in range(n_samples)]
    
    def display(self, ctxs): display(HTML(pd.DataFrame(ctxs).to_html(index=False)))

In [ ]:
path = untar_data(URLs.IMDB_SAMPLE)
df = pd.read_csv(path/'texts.csv')

In [ ]:
df.head()

,label,text,is_valid
0,negative,"Un-bleeping-believable! Meg Ryan doesn't even look her usual pert lovable self in this, which normally makes me forgive her shallow ticky acting schtick. Hard to believe she was the producer on this dog. Plus Kevin Kline: what kind of suicide trip has his career been on? Whoosh... Banzai!!! Finally this was directed by the guy who did Big Chill? Must be a replay of Jonestown - hollywood style. Wooofff!",False
1,positive,"This is a extremely well-made film. The acting, script and camera-work are all first-rate. The music is good, too, though it is mostly early in the film, when things are still relatively cheery. There are no really superstars in the cast, though several faces will be familiar. The entire cast does an excellent job with the script.<br /><br />But it is hard to watch, because there is no good end to a situation like the one presented. It is now fashionable to blame the British for setting Hindus and Muslims against each other, and then cruelly separating them into two countries. There is som...",False
2,negative,"Every once in a long while a movie will come along that will be so awful that I feel compelled to warn people. If I labor all my days and I can save but one soul from watching this movie, how great will be my joy.<br /><br />Where to begin my discussion of pain. For starters, there was a musical montage every five minutes. There was no character development. Every character was a stereotype. We had swearing guy, fat guy who eats donuts, goofy foreign guy, etc. The script felt as if it were being written as the movie was being shot. The production value was so incredibly low that it felt li...",False
3,positive,"Name just says it all. I watched this movie with my dad when it came out and having served in Korea he had great admiration for the man. The disappointing thing about this film is that it only concentrate on a short period of the man's life - interestingly enough the man's entire life would have made such an epic bio-pic that it is staggering to imagine the cost for production.<br /><br />Some posters elude to the flawed characteristics about the man, which are cheap shots. The theme of the movie ""Duty, Honor, Country"" are not just mere words blathered from the lips of a high-brassed offic...",False
4,negative,"This movie succeeds at being one of the most unique movies you've seen. However this comes from the fact that you can't make heads or tails of this mess. It almost seems as a series of challenges set up to determine whether or not you are willing to walk out of the movie and give up the money you just paid. If you don't want to feel slighted you'll sit through this horrible film and develop a real sense of pity for the actors involved, they've all seen better days, but then you realize they actually got paid quite a bit of money to do this and you'll lose pity for them just like you've alr...",False


In [ ]:
df_tok,count = tokenize_df(df, 'text')

In [ ]:
df_tok.head()

,label,is_valid,text,text_lengths
0,negative,False,"xxbos▁xxmaj▁un▁-▁bleeping▁-▁believable▁!▁xxmaj▁meg▁xxmaj▁ryan▁does▁n't▁even▁look▁her▁usual▁pert▁lovable▁self▁in▁this▁,▁which▁normally▁makes▁me▁forgive▁her▁shallow▁ticky▁acting▁schtick▁.▁xxmaj▁hard▁to▁believe▁she▁was▁the▁producer▁on▁this▁dog▁.▁xxmaj▁plus▁xxmaj▁kevin▁xxmaj▁kline▁:▁what▁kind▁of▁suicide▁trip▁has▁his▁career▁been▁on▁?▁xxmaj▁whoosh▁...▁xxmaj▁banzai▁!▁!▁!▁xxmaj▁finally▁this▁was▁directed▁by▁the▁guy▁who▁did▁xxmaj▁big▁xxmaj▁chill▁?▁xxmaj▁must▁be▁a▁replay▁of▁xxmaj▁jonestown▁-▁hollywood▁style▁.▁xxmaj▁wooofff▁!",103.0
1,positive,False,"xxbos▁xxmaj▁this▁is▁a▁extremely▁well▁-▁made▁film▁.▁xxmaj▁the▁acting▁,▁script▁and▁camera▁-▁work▁are▁all▁first▁-▁rate▁.▁xxmaj▁the▁music▁is▁good▁,▁too▁,▁though▁it▁is▁mostly▁early▁in▁the▁film▁,▁when▁things▁are▁still▁relatively▁cheery▁.▁xxmaj▁there▁are▁no▁really▁superstars▁in▁the▁cast▁,▁though▁several▁faces▁will▁be▁familiar▁.▁xxmaj▁the▁entire▁cast▁does▁an▁excellent▁job▁with▁the▁script▁.▁\n\n▁xxmaj▁but▁it▁is▁hard▁to▁watch▁,▁because▁there▁is▁no▁good▁end▁to▁a▁situation▁like▁the▁one▁presented▁.▁xxmaj▁it▁is▁now▁fashionable▁to▁blame▁the▁xxmaj▁british▁for▁setting▁xxmaj▁hindus▁and▁xxmaj▁muslims▁against...",462.0
2,negative,False,"xxbos▁xxmaj▁every▁once▁in▁a▁long▁while▁a▁movie▁will▁come▁along▁that▁will▁be▁so▁awful▁that▁i▁feel▁compelled▁to▁warn▁people▁.▁xxmaj▁if▁i▁labor▁all▁my▁days▁and▁i▁can▁save▁but▁one▁soul▁from▁watching▁this▁movie▁,▁how▁great▁will▁be▁my▁joy▁.▁\n\n▁xxmaj▁where▁to▁begin▁my▁discussion▁of▁pain▁.▁xxmaj▁for▁starters▁,▁there▁was▁a▁musical▁montage▁every▁five▁minutes▁.▁xxmaj▁there▁was▁no▁character▁development▁.▁xxmaj▁every▁character▁was▁a▁stereotype▁.▁xxmaj▁we▁had▁swearing▁guy▁,▁fat▁guy▁who▁eats▁donuts▁,▁goofy▁foreign▁guy▁,▁etc▁.▁xxmaj▁the▁script▁felt▁as▁if▁it▁were▁being▁written▁as▁the▁movie▁was▁being▁shot...",220.0
3,positive,False,"xxbos▁xxmaj▁name▁just▁says▁it▁all▁.▁i▁watched▁this▁movie▁with▁my▁dad▁when▁it▁came▁out▁and▁having▁served▁in▁xxmaj▁korea▁he▁had▁great▁admiration▁for▁the▁man▁.▁xxmaj▁the▁disappointing▁thing▁about▁this▁film▁is▁that▁it▁only▁concentrate▁on▁a▁short▁period▁of▁the▁man▁'s▁life▁-▁interestingly▁enough▁the▁man▁'s▁entire▁life▁would▁have▁made▁such▁an▁epic▁bio▁-▁pic▁that▁it▁is▁staggering▁to▁imagine▁the▁cost▁for▁production▁.▁\n\n▁xxmaj▁some▁posters▁elude▁to▁the▁flawed▁characteristics▁about▁the▁man▁,▁which▁are▁cheap▁shots▁.▁xxmaj▁the▁theme▁of▁the▁movie▁""▁duty▁,▁xxmaj▁honor▁,▁xxmaj▁country▁""▁are▁not▁just▁mer...",184.0
4,negative,False,"xxbos▁xxmaj▁this▁movie▁succeeds▁at▁being▁one▁of▁the▁most▁unique▁movies▁you▁'ve▁seen▁.▁xxmaj▁however▁this▁comes▁from▁the▁fact▁that▁you▁ca▁n't▁make▁heads▁or▁tails▁of▁this▁mess▁.▁xxmaj▁it▁almost▁seems▁as▁a▁series▁of▁challenges▁set▁up▁to▁determine▁whether▁or▁not▁you▁are▁willing▁to▁walk▁out▁of▁the▁movie▁and▁give▁up▁the▁money▁you▁just▁paid▁.▁xxmaj▁if▁you▁do▁n't▁want▁to▁feel▁slighted▁you▁'ll▁sit▁through▁this▁horrible▁film▁and▁develop▁a▁real▁sense▁of▁pity▁for▁the▁actors▁involved▁,▁they▁'ve▁all▁seen▁better▁days▁,▁but▁then▁you▁realize▁they▁actually▁got▁paid▁quite▁a▁bit▁of▁money▁to▁do▁this▁and▁you▁'l...",398.0


In [ ]:
texts,lengths = df_tok['text'].values,df_tok['text_lengths'].values.astype(np.int)

In [ ]:
splits = RandomSplitter()(L(t for t in texts))
dsrc = DataSource(L(t for t in texts), type_tfms=[Numericalize(make_vocab(count))], filts=splits)

In [ ]:
dsrc.decode_at(0)

("xxbos▁xxmaj▁un▁-▁xxunk▁-▁believable▁!▁xxmaj▁meg▁xxmaj▁ryan▁does▁n't▁even▁look▁her▁usual▁xxunk▁lovable▁self▁in▁this▁,▁which▁normally▁makes▁me▁forgive▁her▁shallow▁xxunk▁acting▁xxunk▁.▁xxmaj▁hard▁to▁believe▁she▁was▁the▁producer▁on▁this▁dog▁.▁xxmaj▁plus▁xxmaj▁kevin▁xxmaj▁kline▁:▁what▁kind▁of▁suicide▁trip▁has▁his▁career▁been▁on▁?▁xxmaj▁xxunk▁...▁xxmaj▁xxunk▁!▁!▁!▁xxmaj▁finally▁this▁was▁directed▁by▁the▁guy▁who▁did▁xxmaj▁big▁xxmaj▁xxunk▁?▁xxmaj▁must▁be▁a▁replay▁of▁xxmaj▁jonestown▁-▁hollywood▁style▁.▁xxmaj▁xxunk▁!",)

In [ ]:
bs = 16
ds = LM_PreLoader(dsrc.train, lengths=lengths[splits[0]], bs=bs)
dl = TfmdDL(ds, bs=bs, num_workers=0)

In [ ]:
x,y = dl.one_batch()

In [ ]:
dl.decode_batch((x,y))

(#10) [('xxbos▁xxmaj▁this▁is▁a▁really▁interesting▁movie▁.▁xxmaj▁it▁is▁an▁action▁movie▁with▁comedy▁mixed▁in▁.▁xxmaj▁foxx▁teams▁up▁with▁comedian▁xxmaj▁xxunk▁in▁this▁movie▁to▁give▁it▁a▁comedic▁spin▁.▁xxmaj▁it▁will▁keep▁you▁wondering▁what▁s▁going▁to▁happen▁to▁xxmaj▁foxx▁next▁.▁xxmaj▁it▁was▁a▁well▁shot▁movie▁,▁the▁director▁used▁the▁right▁colors▁in▁this',),('confusion▁and▁disappointment▁on▁their▁faces▁.▁\n\n▁xxmaj▁to▁the▁makers▁of▁this▁movie▁,▁you▁owe▁me▁two▁hours▁.▁xxbos▁a▁man▁xxunk▁to▁life▁alone▁after▁45▁years▁of▁marriage▁.▁xxmaj▁he▁also▁has▁to▁solve▁the▁problem▁of▁the▁family▁xxunk▁xxunk▁,▁xxmaj▁tulip▁,▁which▁refuses▁to▁allow▁itself▁to▁be▁xxunk▁.▁xxmaj▁until▁,▁that▁is▁,▁he▁xxunk▁his',),('xxunk▁,▁and▁xxunk▁knock▁it▁out▁of▁the▁number▁one▁spot▁.▁xxmaj▁every▁time▁i▁hear▁someone▁declare▁"▁titanic▁"▁is▁the▁greatest▁film▁they▁\'ve▁ever▁seen▁,▁i▁think▁to▁myself▁,▁"▁you▁do▁n\'t▁see▁a▁lot▁of▁movies▁,▁do▁you▁?▁"▁xxmaj▁what▁a▁travesty▁.▁xxmaj▁you▁could▁make▁50▁good▁films▁that▁are▁a▁lot▁better▁than▁"',

In [ ]:
ds.decode((x[0],))

('xxbos▁xxmaj▁this▁is▁a▁really▁interesting▁movie▁.▁xxmaj▁it▁is▁an▁action▁movie▁with▁comedy▁mixed▁in▁.▁xxmaj▁foxx▁teams▁up▁with▁comedian▁xxmaj▁xxunk▁in▁this▁movie▁to▁give▁it▁a▁comedic▁spin▁.▁xxmaj▁it▁will▁keep▁you▁wondering▁what▁s▁going▁to▁happen▁to▁xxmaj▁foxx▁next▁.▁xxmaj▁it▁was▁a▁well▁shot▁movie▁,▁the▁director▁used▁the▁right▁colors▁in▁this',)

### Showing

In [ ]:
dsrc = DataSource(L(t for t in texts), type_tfms=[Numericalize(make_vocab(count))], filts=splits)
bs = 16
ds = LM_PreLoader(dsrc.train, lengths=lengths[splits[0]], bs=bs)
dl = TfmdDL(ds, bs=bs, num_workers=0)

In [ ]:
dl.show_batch()

index,text
0,"xxbos▁xxmaj▁this▁is▁a▁really▁interesting▁movie▁.▁xxmaj▁it▁is▁an▁action▁movie▁with▁comedy▁mixed▁in▁.▁xxmaj▁foxx▁teams▁up▁with▁comedian▁xxmaj▁xxunk▁in▁this▁movie▁to▁give▁it▁a▁comedic▁spin▁.▁xxmaj▁it▁will▁keep▁you▁wondering▁what▁s▁going▁to▁happen▁to▁xxmaj▁foxx▁next▁.▁xxmaj▁it▁was▁a▁well▁shot▁movie▁,▁the▁director▁used▁the▁right▁colors▁in▁this"
1,"confusion▁and▁disappointment▁on▁their▁faces▁.▁\n\n▁xxmaj▁to▁the▁makers▁of▁this▁movie▁,▁you▁owe▁me▁two▁hours▁.▁xxbos▁a▁man▁xxunk▁to▁life▁alone▁after▁45▁years▁of▁marriage▁.▁xxmaj▁he▁also▁has▁to▁solve▁the▁problem▁of▁the▁family▁xxunk▁xxunk▁,▁xxmaj▁tulip▁,▁which▁refuses▁to▁allow▁itself▁to▁be▁xxunk▁.▁xxmaj▁until▁,▁that▁is▁,▁he▁xxunk▁his"
2,"xxunk▁,▁and▁xxunk▁knock▁it▁out▁of▁the▁number▁one▁spot▁.▁xxmaj▁every▁time▁i▁hear▁someone▁declare▁""▁titanic▁""▁is▁the▁greatest▁film▁they▁'ve▁ever▁seen▁,▁i▁think▁to▁myself▁,▁""▁you▁do▁n't▁see▁a▁lot▁of▁movies▁,▁do▁you▁?▁""▁xxmaj▁what▁a▁travesty▁.▁xxmaj▁you▁could▁make▁50▁good▁films▁that▁are▁a▁lot▁better▁than▁"""
3,"bad▁-▁guy▁-▁like▁things▁,▁just▁in▁case▁you▁did▁n't▁quite▁catch▁on▁.▁xxmaj▁the▁good▁guy▁does▁brave▁and▁noble▁things▁just▁in▁case▁you▁did▁n't▁know▁he▁was▁the▁good▁guy▁.▁xxmaj▁and▁oh▁lord▁,▁the▁plane▁scene▁.▁""▁my▁xxup▁husband▁xxup▁is▁xxup▁missing▁!▁xxup▁oh▁xxup▁my▁xxup▁god▁!▁xxup▁it▁'s▁xxup▁impossible▁xxup"
4,",▁unrealistic▁dialogue▁.▁xxmaj▁that▁it▁got▁as▁many▁positive▁votes▁suggests▁some▁people▁may▁be▁joking▁.▁xxbos▁an▁very▁good▁storyline▁,▁good▁xxunk▁to▁it▁...▁but▁the▁10▁last▁seconds▁destroyed▁the▁whole▁movie▁...▁what▁happened▁?▁extremely▁well▁made▁and▁an▁good▁story▁destroyed▁in▁the▁last▁seconds▁...▁sorry▁to▁say▁but▁a▁1▁in▁vote▁...▁that▁s▁what▁it▁it"
5,models▁.▁xxmaj▁even▁the▁shooting▁was▁bad▁-▁in▁one▁scene▁the▁xxmaj▁xxunk▁xxunk▁his▁six▁-▁shooter▁about▁nine▁times▁and▁could▁n't▁hit▁any▁of▁three▁large▁men▁who▁were▁only▁about▁twenty▁feet▁away▁.▁i▁had▁to▁turn▁it▁off▁after▁about▁15▁minutes▁of▁this▁xxunk▁.▁xxmaj▁perhaps▁those▁who▁xxunk▁in▁this▁movie▁could▁have▁taken▁some▁lessons▁at▁the▁xxmaj
6,"and▁never▁develops▁(▁just▁because▁the▁subject▁matter▁is▁deep▁,▁does▁n't▁mean▁the▁story▁is▁good▁)▁.▁a▁good▁story▁has▁dynamic▁characters▁.▁a▁dynamic▁character▁is▁one▁that▁experiences▁a▁major▁character▁change▁,▁and▁is▁xxunk▁for▁that▁change▁over▁the▁course▁of▁the▁movie▁.▁xxmaj▁in▁xxmaj▁georgia▁xxmaj▁rule▁,▁the▁character▁changes▁were▁abrupt▁and▁undeveloped▁.▁xxmaj▁secondly"
7,"project▁with▁its▁shaky▁camera▁work▁and▁xxmaj▁the▁xxmaj▁texas▁xxmaj▁chainsaw▁xxmaj▁massacre▁in▁its▁bare▁-▁bones▁approach▁to▁the▁material▁but▁succeeds▁at▁being▁neither▁.▁xxmaj▁what▁we▁get▁instead▁are▁xxunk▁scenes▁of▁chase▁,▁torture▁,▁release▁;▁chase▁torture▁,▁release▁,▁in▁that▁order▁for▁xxunk▁minutes▁with▁long▁interludes▁of▁xxunk▁,▁xxunk▁,▁and▁incoherent▁rambling▁acting▁as▁plenty"
8,"now▁and▁rent▁it▁!▁(▁do▁n't▁forget▁popcorn▁!▁)▁xxbos▁i▁saw▁this▁movie▁years▁ago▁on▁late▁night▁television▁.▁xxmaj▁back▁then▁it▁went▁by▁the▁title▁of▁""▁xxunk▁to▁xxmaj▁heaven▁""▁.▁xxmaj▁even▁as▁a▁young▁boy▁,▁i▁remember▁being▁deeply▁moved▁by▁the▁story▁and▁xxunk▁by▁the▁visual▁effects▁of▁the▁court▁trial▁(▁those▁who▁have▁seen"
9,"are▁interesting▁and▁have▁some▁historical▁value▁as▁evidence▁of▁what▁this▁environment▁looked▁like▁in▁1930▁.▁xxmaj▁the▁shots▁of▁the▁xxmaj▁xxunk▁and▁other▁natives▁are▁also▁interesting▁,▁although▁these▁suffer▁from▁the▁xxmaj▁johnsons▁'▁xxunk▁to▁stage▁events▁in▁a▁manner▁that▁makes▁the▁natives▁look▁'▁wild▁'▁and▁alien▁.▁\n\n▁xxmaj▁the▁best▁(▁and▁funniest▁)▁scene▁in▁'▁congorilla"


## Classification

In [ ]:
def pad_collate(samples, pad_idx=1, pad_first=True, backwards=False):
    "Function that collect samples and adds padding. Flips token order if needed"
    max_len = max([len(s[0]) for s in samples])
    res = torch.zeros(len(samples), max_len).long() + pad_idx
    if backwards: pad_first = not pad_first
    for i,s in enumerate(samples):
        if pad_first: res[i,-len(s[0]):] = LongTensor(s[0])
        else:         res[i,:len(s[0]):] = LongTensor(s[0])
    if backwards: res = res.flip(1)
    return TensorTextBase(res), tensor(np.array([s[1] for s in samples]))

In [ ]:
splits = RandomSplitter()(range(len(df)))
_get_txt = lambda i: df_tok["text"][i]
_get_lbl = lambda i: df_tok["label"][i]
dsrc = DataSource(range(len(df)), type_tfms=[[_get_txt, Numericalize(make_vocab(count))], [_get_lbl, Categorize()]], filts=splits)
dl = TfmdDL(dsrc, collate_fn=TfmdCollate(collate_fn=pad_collate))

In [ ]:
dl.show_batch()

index,text,category
0,"xxbos▁xxmaj▁un▁-▁xxunk▁-▁believable▁!▁xxmaj▁meg▁xxmaj▁ryan▁does▁n't▁even▁look▁her▁usual▁xxunk▁lovable▁self▁in▁this▁,▁which▁normally▁makes▁me▁forgive▁her▁shallow▁xxunk▁acting▁xxunk▁.▁xxmaj▁hard▁to▁believe▁she▁was▁the▁producer▁on▁this▁dog▁.▁xxmaj▁plus▁xxmaj▁kevin▁xxmaj▁kline▁:▁what▁kind▁of▁suicide▁trip▁has▁his▁career▁been▁on▁?▁xxmaj▁xxunk▁...▁xxmaj▁xxunk▁!▁!▁!▁xxmaj▁finally▁this▁was▁directed▁by▁the▁guy▁who▁did▁xxmaj▁big▁xxmaj▁xxunk▁?▁xxmaj▁must▁be▁a▁replay▁of▁xxmaj▁jonestown▁-▁hollywood▁style▁.▁xxmaj▁xxunk▁!",negative
1,"xxbos▁xxmaj▁this▁is▁a▁extremely▁well▁-▁made▁film▁.▁xxmaj▁the▁acting▁,▁script▁and▁camera▁-▁work▁are▁all▁first▁-▁rate▁.▁xxmaj▁the▁music▁is▁good▁,▁too▁,▁though▁it▁is▁mostly▁early▁in▁the▁film▁,▁when▁things▁are▁still▁relatively▁xxunk▁.▁xxmaj▁there▁are▁no▁really▁xxunk▁in▁the▁cast▁,▁though▁several▁faces▁will▁be▁familiar▁.▁xxmaj▁the▁entire▁cast▁does▁an▁excellent▁job▁with▁the▁script▁.▁\n\n▁xxmaj▁but▁it▁is▁hard▁to▁watch▁,▁because▁there▁is▁no▁good▁end▁to▁a▁situation▁like▁the▁one▁presented▁.▁xxmaj▁it▁is▁now▁xxunk▁to▁blame▁the▁xxmaj▁british▁for▁setting▁xxmaj▁hindus▁and▁xxmaj▁muslims▁against▁each▁other▁,▁and▁then▁xxunk▁xxunk▁them▁into▁two▁countries▁.▁xxmaj▁there▁is▁some▁merit▁in▁this▁view▁,▁but▁it▁'s▁also▁true▁that▁no▁one▁forced▁xxmaj▁hindus▁and▁xxmaj▁muslims▁in▁the▁region▁to▁xxunk▁each▁other▁as▁they▁did▁around▁the▁time▁of▁partition▁.▁xxmaj▁it▁seems▁more▁likely▁that▁the▁xxmaj▁british▁simply▁saw▁the▁xxunk▁between▁the▁xxunk▁and▁were▁clever▁enough▁to▁exploit▁them▁to▁their▁own▁ends▁.▁\n\n▁xxmaj▁the▁result▁is▁that▁there▁is▁much▁cruelty▁and▁inhumanity▁in▁the▁situation▁and▁this▁is▁very▁unpleasant▁to▁remember▁and▁to▁see▁on▁the▁screen▁.▁xxmaj▁but▁it▁is▁never▁painted▁as▁a▁black▁-▁and▁-▁white▁case▁.▁xxmaj▁there▁is▁xxunk▁and▁xxunk▁on▁both▁sides▁,▁and▁also▁the▁hope▁for▁change▁in▁the▁younger▁generation▁.▁\n\n▁xxmaj▁there▁is▁redemption▁of▁a▁sort▁,▁in▁the▁end▁,▁when▁xxmaj▁xxunk▁has▁to▁make▁a▁hard▁choice▁between▁a▁man▁who▁has▁ruined▁her▁life▁,▁but▁also▁truly▁loved▁her▁,▁and▁her▁family▁which▁has▁xxunk▁her▁,▁then▁later▁come▁looking▁for▁her▁.▁xxmaj▁but▁by▁that▁point▁,▁she▁has▁no▁xxunk▁that▁is▁without▁great▁pain▁for▁her▁.▁\n\n▁xxmaj▁this▁film▁carries▁the▁message▁that▁both▁xxmaj▁muslims▁and▁xxmaj▁hindus▁have▁their▁grave▁faults▁,▁and▁also▁that▁both▁can▁be▁xxunk▁and▁caring▁people▁.▁xxmaj▁the▁reality▁of▁partition▁makes▁that▁xxunk▁all▁the▁more▁wrenching▁,▁since▁there▁can▁never▁be▁real▁xxunk▁across▁the▁xxmaj▁india▁/▁xxmaj▁pakistan▁border▁.▁xxmaj▁in▁that▁sense▁,▁it▁is▁similar▁to▁""▁mr▁&▁xxmaj▁mrs▁xxmaj▁xxunk▁""▁.▁\n\n▁xxmaj▁in▁the▁end▁,▁we▁were▁glad▁to▁have▁seen▁the▁film▁,▁even▁though▁the▁resolution▁was▁heartbreaking▁.▁xxmaj▁if▁the▁xxup▁uk▁and▁xxup▁us▁could▁deal▁with▁their▁own▁xxunk▁of▁racism▁with▁this▁kind▁of▁xxunk▁,▁they▁would▁certainly▁be▁better▁off▁.",positive
2,"xxbos▁xxmaj▁every▁once▁in▁a▁long▁while▁a▁movie▁will▁come▁along▁that▁will▁be▁so▁awful▁that▁i▁feel▁compelled▁to▁warn▁people▁.▁xxmaj▁if▁i▁labor▁all▁my▁days▁and▁i▁can▁save▁but▁one▁soul▁from▁watching▁this▁movie▁,▁how▁great▁will▁be▁my▁joy▁.▁\n\n▁xxmaj▁where▁to▁begin▁my▁discussion▁of▁pain▁.▁xxmaj▁for▁xxunk▁,▁there▁was▁a▁musical▁montage▁every▁five▁minutes▁.▁xxmaj▁there▁was▁no▁character▁development▁.▁xxmaj▁every▁character▁was▁a▁stereotype▁.▁xxmaj▁we▁had▁xxunk▁guy▁,▁fat▁guy▁who▁eats▁donuts▁,▁goofy▁foreign▁guy▁,▁etc▁.▁xxmaj▁the▁script▁felt▁as▁if▁it▁were▁being▁written▁as▁the▁movie▁was▁being▁shot▁.▁xxmaj▁the▁production▁value▁was▁so▁incredibly▁low▁that▁it▁felt▁like▁i▁was▁watching▁a▁junior▁high▁video▁presentation▁.▁xxmaj▁have▁the▁directors▁,▁producers▁,▁etc▁.▁ever▁even▁seen▁a▁movie▁before▁?▁xxmaj▁xxunk▁is▁getting▁worse▁and▁worse▁with▁every▁new▁entry▁.▁xxmaj▁the▁concept▁for▁this▁movie▁sounded▁so▁funny▁.▁xxmaj▁how▁could▁you▁go▁wrong▁with▁xxmaj▁gary▁xxmaj▁coleman▁and▁a▁handful▁of▁somewhat▁legitimate▁actors▁.▁xxmaj▁but▁trust▁me▁when▁i▁say▁this▁,▁things▁went▁wrong▁,▁xxup▁very▁xxup▁wrong▁.",negative
3,"xxbos▁xxmaj▁name▁just▁says▁it▁all▁.▁i▁watched▁this▁movie▁with▁my▁dad▁when▁it▁came▁out▁a